In [79]:
%load_ext autoreload
%autoreload 2

import os
from datetime import datetime
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.utils.tensorboard import SummaryWriter

from market_dynamics import bs_delta, bs_generator, bs_call_price
from data import SimulationData
from utils import call_payoff, stochastic_integral
from models import ControlNet
from train import train

current_time = datetime.now().strftime('%b%d_%H-%M-%S')
comment = ""
log_dir = os.path.join('experiments', current_time + '_' + comment)
writer = SummaryWriter(log_dir)

n_simulations = 100
initial_value = 100
sigma = 0.2
n_steps = 50
rf = 0
strike = 95
fc_dims = [16, 16]
LR = 0.01
EPOCHS = 5

bs_params = {"n_simulations": n_simulations,
             "n_steps": n_steps,
             "initial_value": initial_value,
             "sigma": sigma,
             }

price_params = {"n_steps": n_steps,
                "initial_value": initial_value,
                "sigma": sigma,
                "rf": rf,
                "strike": strike,
                }

payoff_params = {"strike": strike}


dt = SimulationData(bs_generator, bs_params, bs_call_price, price_params, call_payoff, payoff_params)
data_loader = DataLoader(dt, batch_size=20, shuffle=True)
cn = ControlNet(n_steps, 1, fc_dims, 1)

optimizer = Adam(cn.parameters(), lr=LR)
criterion = torch.nn.MSELoss()
train(data_loader, cn, criterion , optimizer, EPOCHS, writer)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Epoch 0: : 5batch [00:00, 26.00batch/s, loss=16]
Epoch 1: : 5batch [00:00, 27.71batch/s, loss=13.1]
Epoch 2: : 5batch [00:00, 19.71batch/s, loss=5.64]
Epoch 3: : 5batch [00:00, 13.69batch/s, loss=7.09]
Epoch 4: : 5batch [00:00, 16.03batch/s, loss=2.54]


In [80]:
def delta(x):
    return bs_delta(n_steps, x, sigma, rf, strike)

In [81]:
# Evaluation

_ ,model_loss = test(data_loader, cn, criterion)

l = len(data_loader.dataset)
x, x_inc, payoff, price = data_loader.dataset[:l]

d_hedge = x.apply_(delta)
si = stochastic_integral(x_inc, d_hedge)
delta_loss = criterion(price + si, payoff)

print(model_loss)
print(delta_loss)

tensor(1.8278, grad_fn=<MseLossBackward>)
tensor(2.8380)
